第一次使用 `salem` 需要在 `home` 目录中创建 `.salem_cache` 文件夹，并下载 `salem-sample-data` 的数据包。为了更友好地直接导入 `salem` ，本项目已挂载社区的共享数据集[salem_cache文件集合](https://www.heywhale.com/mw/dataset/64716032b1251d62bfb38aa8/file)，再把该数据集的数据复制到 `/home/mw/.salem_cache/`  内，实现手动下载，避免卡住。

In [2]:
import os
import os.path

if(not os.path.exists('/root/.salem_cache')):
    os.mkdir('/root/.salem_cache') # 创建后它隐形的

! cp -r ./dataset/salem_cache /root/.salem_cache/ # 拷贝文件
! ls -ll /root/.salem_cache/  # 检查有无成功

total 52908
drwxr-xr-x 5 root root     4096 Jun 18 06:03 cache
drwxr-xr-x 2 root root     4096 Jun 18 05:50 downloads
drwxr-xr-x 6 root root     4096 Jun 18 05:53 salem-sample-data-57e6d694aa470b967336f5ca2d4fc743c5c8efd6
-rw-r--r-- 1 root root 54159650 Jun 18 05:53 salem-sample-data-57e6d694aa470b967336f5ca2d4fc743c5c8efd6.zip
drwxr-xr-x 1 root root     4096 Jun 18 05:50 salem_cache


## 一次性打开多个wrfout文件，时间维度位移后相减获得累计降水

In [34]:
import salem

fcst_dir = "./dataset/wrfout"
# 可以传入字符串
wrf = salem.open_mf_wrf_dataset( f"{fcst_dir}/wrfout_d01*" ,preprocess = lambda x: x[['RAINC','RAINSH','RAINNC']] )

# 也可以传入具体的wrfout文件名列表
# fn1 = f"{fcst_dir}/wrfout_d01_2016-07-12_00:00:00"
# fn2 = f"{fcst_dir}/wrfout_d01_2016-07-12_03:00:00"
# wrf = salem.open_mf_wrf_dataset( [fn1, fn2] ,preprocess = lambda x: x[['RAINC', 'RAINNC']] )

pcp = wrf.diff(dim="time", n=1)
# 或者
#pcp = wrf - wrf.shift(time=1)
pcp

<xarray.Dataset>
Dimensions:      (time: 12, south_north: 437, west_east: 447)
Coordinates:
    lat          (south_north, west_east) float32 20.89 20.89 ... 32.78 32.78
    lon          (south_north, west_east) float32 116.5 116.6 ... 130.2 130.3
    xtime        (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2019-08-08T19:00:00 ... 2019-08-09T06:...
  * west_east    (west_east) float64 -6.69e+05 -6.66e+05 ... 6.66e+05 6.69e+05
  * south_north  (south_north) float64 -6.54e+05 -6.51e+05 ... 6.51e+05 6.54e+05
Data variables:
    RAINC        (time, south_north, west_east) float32 dask.array<chunksize=(1, 437, 447), meta=np.ndarray>
    RAINSH       (time, south_north, west_east) float32 dask.array<chunksize=(1, 437, 447), meta=np.ndarray>
    RAINNC       (time, south_north, west_east) float32 dask.array<chunksize=(1, 437, 447), meta=np.ndarray>
Attributes: (12/135)
    TITLE:                            OUTPUT FROM WRF V4.4.2 MODEL
    START_DATE:                      2019-08-08_18:00:00
    SIMULATION_START_DATE:           2019-08-08_18:00:00
    WEST-EAST_GRID_DIMENSION:        448
    SOUTH-NORTH_GRID_DIMENSION:      438
    BOTTOM-TOP_GRID_DIMENSION:       35
    ...                              ...
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2
    pyproj_srs:                      +proj=lcc +lat_0=27 +lon_0=123 +lat_1=20...

In [35]:
pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 dask.array<chunksize=(), meta=np.ndarray>
    RAINSH   float32 dask.array<chunksize=(), meta=np.ndarray>
    RAINNC   float32 dask.array<chunksize=(), meta=np.ndarray>

## 打开两个文件，前后相减获得累计降水

In [39]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-08_18_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-08_19_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 69.97

In [40]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-08_19_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-08_20_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 139.9

In [41]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-08_20_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-08_21_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 125.0

In [42]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-08_21_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-08_22_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 72.84

In [43]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-08_22_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-08_23_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 75.63

In [44]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-08_23_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-09_00_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 77.45

In [46]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-09_00_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-09_01_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 121.3

In [47]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-09_01_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-09_02_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 107.7

In [48]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-09_02_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-09_03_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 87.8

In [49]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-09_03_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-09_04_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 71.51

In [50]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-09_04_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-09_05_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 96.11

In [51]:
import salem

fcst_dir = "./dataset/wrfout"
fn1 = f"{fcst_dir}/wrfout_d01_2019-08-09_05_00_00"
fn2 = f"{fcst_dir}/wrfout_d01_2019-08-09_06_00_00"

ds1 = salem.open_wrf_dataset(fn1)
ds1 = ds1[['RAINC','RAINSH','RAINNC']].isel(time=0)

ds2 = salem.open_wrf_dataset(fn2)
ds2 = ds2[['RAINC','RAINSH','RAINNC']].isel(time=0)

pcp = ds2 - ds1 

pcp.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    RAINC    float32 0.0
    RAINSH   float32 0.0
    RAINNC   float32 94.07

## WRF模拟降水可视化

In [ ]:
import salem 
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader

fn = "./dataset/wrfout/wrfout_d01_2019-08-09_06_00_00"
ds = salem.open_wrf_dataset(fn)
pcp = ds.RAINC + ds.RAINNC

rgb = ([237, 237, 237],[209, 209, 209],[173, 173, 173],[131, 131, 131],
        [93 , 93 , 93],[151, 198, 223],[111, 176, 214],[49 , 129, 189],
        [26 , 104, 174],[8  , 79 , 153],[62 , 168, 91],[110, 193, 115],
        [154, 214, 149],[192, 230, 185],[223, 242, 217],[255, 255, 164],
        [255, 243, 0],[255, 183, 0],[255, 123, 0],[255, 62 , 0],
        [255, 2  , 0],[196, 0  , 0],[136, 0  , 0],
)
clors = np.array(rgb)/255.
clevs = [0.1, 1,  2,  5, 7.5, 10, 13, 16, 20, 25, 30, 35, 40, 50, 60,
         70, 80, 90, 100, 125, 150, 175, 200, 250
          ]

myproj = ccrs.PlateCarree()
fig = plt.figure(figsize=(16,9.6), dpi=600)
ax = fig.add_subplot(111, projection = myproj )

cf = ax.contourf(pcp.lon, pcp.lat, pcp.isel(time=0), clevs, colors=clors, transform = myproj)
cb_cf = fig.colorbar(cf, ax=ax,ticks=clevs, shrink=.65)

ax.add_geometries(Reader('./dataset/中国地图').geometries(),
                    crs=myproj, facecolor='none',edgecolor='k',linewidth=1)